In [1]:
%env CUDA_DEVICE_ORDER PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES 0

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0


In [2]:
# !mv ~/stackx_base_with_aux_ep_051_val_perplexity_4.74_val_spearman_0.33.pth uploaded/stackx-bert-base/


In [2]:
import pandas as pd
import numpy as np

import seaborn as sns 
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, confusion_matrix
import re
from tqdm import tqdm_notebook
from pathlib import Path

from transformers import BertTokenizer, BertConfig
from pytorch_transformers import BertForSequenceClassification, PretrainedConfig
from multiprocessing import Pool, cpu_count

import json
import torch
import pprint
import numpy as np
import itertools
import functools
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, Dataset
from poutyne.framework import EarlyStopping

from multiprocessing import Pool, cpu_count
from torch.utils.data import Dataset
from tqdm import tqdm
from pathlib import Path
import numpy as np
import pandas as pd
import os
import torch
from typing import List, Text
from transformers import PreTrainedTokenizer

from data import (QuestDataset,
                  ALL_TARGETS,
                  QUESTION_TARGETS,
                  EASY_ANSWER_TARGETS,
                  EASY_QUESTION_TARGETS,
                  HARD_ANSWER_TARGETS, 
                  HARD_QUESTION_TARGETS)
from data.augmentation import BertRandomTokenizer
from metrics import Spearman, spearman_metric
from callbacks import LosswiseCallback, CSVParamLogger
from utils import transform_target_columns_to_ordinals
from models import BertForQuestRegression
from transformers import BertTokenizer

from matplotlib import pyplot as plt
%matplotlib inline

//anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
I0224 13:15:21.951596 4573377984 file_utils.py:37] PyTorch version 1.3.0.post2 available.
I0224 13:15:25.412374 4573377984 file_utils.py:51] TensorFlow version 2.0.0 available.


ModuleNotFoundError: No module named 'pytorch_transformers'

In [4]:
train_df = pd.read_csv('train.csv')

In [5]:
from transformers import BertConfig
tokenizer = BertTokenizer('uploaded/stackx-bert-base/stackx-base-cased-vocab.txt', do_lower_case=False)

def get_model():
    config = BertConfig.from_json_file('uploaded/stackx-bert-base/stackx-base-cased-config.json')
    config.__dict__['num_labels'] = len(ALL_TARGETS)
    
    model = BertForQuestRegression(config)
    state_dict = torch.load('uploaded/stackx-bert-base/stackx_base_with_aux_ep_080_val_perplexity_4.39_val_spearman_0.34.pth',
                   map_location='cpu'
        ) 
    state_dict = {name.replace('LayerNorm.gamma', 'LayerNorm.weight').replace('LayerNorm.beta', 'LayerNorm.bias'): weight
                  for name, weight in state_dict.items()}
    
    del state_dict['classifier.weight']
    del state_dict['classifier.bias']
    model.load_state_dict(state_dict, strict = False)
    return model

In [6]:
from metrics import Spearman

In [7]:
folds = pd.read_csv('data/folds.csv')['fold']

In [8]:
import os
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from poutyne.framework import Model, ModelCheckpoint, CSVLogger
from transformers import AdamW
from torch.optim import Adam


loader_kws = dict(batch_size=4, num_workers=1)


checkpoint_dir = Path('/storage/monty/quest_challenge/stackx_with_aux_80/')

if not checkpoint_dir.exists():
    os.makedirs(str(checkpoint_dir))

In [ ]:
spearman = Spearman(ALL_TARGETS)
for fold_idx in range(5):
    train_frame, val_frame = train_df[folds != fold_idx], train_df[folds == fold_idx]

    train_dataset = QuestDataset(train_frame, tokenizer, max_seg_length=512)
    val_dataset = QuestDataset(val_frame, tokenizer, max_seg_length=512)

    train_loader = DataLoader(train_dataset, shuffle=True, **loader_kws)
    val_loader = DataLoader(val_dataset, shuffle=False, **loader_kws)
    
    model = get_model()
    trainer = Model(model, Adam(model.parameters(), lr = 3e-5), 'bce_with_logits', epoch_metrics=[spearman])
    trainer.to('cuda')
    
    
    checkpoint_name = 'stackx_with_aux_80_fold_'+str(fold_idx)+'_ep_{epoch:03}_val_spearman_{val_spearman:.2f}.pth'
    callbacks =[
        spearman.callback,
        ModelCheckpoint(str(checkpoint_dir / checkpoint_name), atomic_write=False),
        CSVParamLogger(str(checkpoint_dir / 'training_log.csv'), append= fold_idx!=0 , extra_metrics=ALL_TARGETS)
    ]
    
    history = trainer.fit_generator(
        train_loader,
        val_loader,
        epochs=8,
        callbacks = callbacks, 
    )
    model.to('cpu')
    del model
    torch.cuda.empty_cache()

Epoch 1/8 415.85s Step 1221/1221: loss: 0.384714, spearman: 0.297420, val_loss: 0.376921, val_spearman: 0.374295
Epoch 2/8 405.40s Step 1221/1221: loss: 0.358942, spearman: 0.396235, val_loss: 0.371795, val_spearman: 0.397263
Epoch 3/8 410.63s Step 1221/1221: loss: 0.345954, spearman: 0.446971, val_loss: 0.370101, val_spearman: 0.404358
Epoch 4/8 410.17s Step 1221/1221: loss: 0.332468, spearman: 0.494726, val_loss: 0.375871, val_spearman: 0.397839
Epoch 5/8 410.22s Step 1221/1221: loss: 0.318069, spearman: 0.542985, val_loss: 0.379286, val_spearman: 0.394113
Epoch 6/8 406.52s Step 1221/1221: loss: 0.303217, spearman: 0.585413, val_loss: 0.391867, val_spearman: 0.383968
Epoch 7/8 407.00s Step 1221/1221: loss: 0.291089, spearman: 0.617667, val_loss: 0.402429, val_spearman: 0.376874
Epoch 8/8 416.18s Step 1221/1221: loss: 0.282086, spearman: 0.642389, val_loss: 0.400215, val_spearman: 0.380890
Epoch 1/8 421.11s Step 1220/1220: loss: 0.387219, spearman: 0.289662, val_loss: 0.371856, val_sp

In [9]:
!ls /storage/monty/quest_challenge/stackx_with_aux/

stackx_with_aux_fold_0_ep_001_val_spearman_0.38.pth
stackx_with_aux_fold_0_ep_002_val_spearman_0.40.pth
stackx_with_aux_fold_0_ep_003_val_spearman_0.40.pth
stackx_with_aux_fold_0_ep_004_val_spearman_0.40.pth
stackx_with_aux_fold_0_ep_005_val_spearman_0.39.pth
stackx_with_aux_fold_0_ep_006_val_spearman_0.39.pth
stackx_with_aux_fold_0_ep_007_val_spearman_0.38.pth
stackx_with_aux_fold_0_ep_008_val_spearman_0.37.pth
stackx_with_aux_fold_1_ep_001_val_spearman_0.38.pth
stackx_with_aux_fold_1_ep_002_val_spearman_0.40.pth
stackx_with_aux_fold_1_ep_003_val_spearman_0.40.pth
stackx_with_aux_fold_1_ep_004_val_spearman_0.40.pth
stackx_with_aux_fold_1_ep_005_val_spearman_0.39.pth
stackx_with_aux_fold_1_ep_006_val_spearman_0.39.pth
stackx_with_aux_fold_1_ep_007_val_spearman_0.38.pth
stackx_with_aux_fold_1_ep_008_val_spearman_0.38.pth
stackx_with_aux_fold_2_ep_001_val_spearman_0.36.pth
stackx_with_aux_fold_2_ep_002_val_spearman_0.39.pth
stackx_with_aux_fold_2_ep_003_val_spearman_0.39.pth
stackx_with_

In [12]:
!mkdir /storage/monty/quest_challenge/submits

In [28]:
!mkdir /storage/monty/quest_challenge/submits/stackx_aux_ep_005_submit

In [29]:
!cp /storage/monty/quest_challenge/stackx_with_aux/*ep_005* /storage/monty/quest_challenge/submits/stackx_aux_ep_005_submit

In [31]:
!cp uploaded/stackx-bert-base/stackx-base-cased-config.json /storage/monty/quest_challenge/submits/stackx_aux_ep_005_submit/

In [15]:
summary = pd.read_csv('/storage/monty/quest_challenge/stackx_with_aux_80/training_log.csv').groupby('epoch').mean()

In [16]:
summary

,time,lr,loss,spearman,val_loss,val_spearman,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
epoch,,,,,,,,,,,,,,,,,,,,,
1,413.262449,0.00003,0.386173,0.297299,0.371963,0.376791,0.345527,0.614576,0.392163,0.272770,...,0.484610,0.169387,0.379546,0.071851,0.111699,0.263339,0.749182,0.269192,0.676294,0.120631
2,410.816827,0.00003,0.358977,0.402029,0.368151,0.398070,0.374610,0.624733,0.404160,0.297390,...,0.510031,0.231108,0.399606,0.138805,0.161617,0.328969,0.760578,0.286568,0.684135,0.163946
3,411.284624,0.00003,0.345241,0.453302,0.368524,0.400611,0.379657,0.636339,0.404208,0.273260,...,0.506188,0.255536,0.408861,0.155035,0.183971,0.337509,0.758290,0.284228,0.681036,0.160476
4,410.717383,0.00003,0.331346,0.502631,0.373107,0.393716,0.361959,0.628755,0.401346,0.266957,...,0.499248,0.245255,0.397475,0.158187,0.174373,0.333453,0.752996,0.271340,0.677399,0.152197
5,409.851900,0.00003,0.316308,0.549825,0.379911,0.387125,0.353607,0.629211,0.399839,0.240614,...,0.494678,0.249538,0.403154,0.157577,0.183019,0.332985,0.743429,0.259349,0.664555,0.156798
6,409.102902,0.00003,0.301744,0.591566,0.389039,0.379273,0.352935,0.626970,0.393321,0.228109,...,0.481139,0.230063,0.390132,0.140585,0.160397,0.315007,0.740578,0.229250,0.670173,0.151370
7,409.644925,0.00003,0.290059,0.623470,0.397447,0.376278,0.343656,0.625906,0.388901,0.224848,...,0.481518,0.235266,0.384552,0.139922,0.170470,0.330018,0.739412,0.216687,0.666764,0.154216
8,412.065767,0.00003,0.281307,0.646360,0.402349,0.373482,0.326928,0.627559,0.383275,0.228420,...,0.476666,0.227535,0.385765,0.133732,0.167929,0.321699,0.738979,0.221496,0.672977,0.156306


In [24]:
pd.Series(summary.values.argmax(0), index=summary.columns)

time                                     0
lr                                       0
loss                                     0
spearman                                 7
val_loss                                 7
val_spearman                             2
question_asker_intent_understanding      2
question_body_critical                   2
question_conversational                  2
question_expect_short_answer             1
question_fact_seeking                    2
question_has_commonly_accepted_answer    1
question_interestingness_others          2
question_interestingness_self            2
question_multi_intent                    2
question_not_really_a_question           3
question_opinion_seeking                 1
question_type_choice                     2
question_type_compare                    3
question_type_consequence                3
question_type_definition                 1
question_type_entity                     3
question_type_instructions               1
question_ty

In [29]:
pd.concat([summary.T[3], summary.max()], axis=1)

,3,0
time,411.284624,413.262449
lr,0.000030,0.000030
loss,0.345241,0.386173
spearman,0.453302,0.646360
val_loss,0.368524,0.402349
val_spearman,0.400611,0.400611
question_asker_intent_understanding,0.379657,0.379657
question_body_critical,0.636339,0.636339
question_conversational,0.404208,0.404208
question_expect_short_answer,0.273260,0.297390


In [31]:
summary.max()[ALL_TARGETS].mean()

0.4036654079056681

In [1]:
!ls /storage/monty/quest_challenge/stackx_with_aux_80/

stackx_with_aux_80_fold_0_ep_001_val_spearman_0.37.pth
stackx_with_aux_80_fold_0_ep_002_val_spearman_0.40.pth
stackx_with_aux_80_fold_0_ep_003_val_spearman_0.40.pth
stackx_with_aux_80_fold_0_ep_004_val_spearman_0.40.pth
stackx_with_aux_80_fold_0_ep_005_val_spearman_0.39.pth
stackx_with_aux_80_fold_0_ep_006_val_spearman_0.38.pth
stackx_with_aux_80_fold_0_ep_007_val_spearman_0.38.pth
stackx_with_aux_80_fold_0_ep_008_val_spearman_0.38.pth
stackx_with_aux_80_fold_1_ep_001_val_spearman_0.38.pth
stackx_with_aux_80_fold_1_ep_002_val_spearman_0.41.pth
stackx_with_aux_80_fold_1_ep_003_val_spearman_0.41.pth
stackx_with_aux_80_fold_1_ep_004_val_spearman_0.40.pth
stackx_with_aux_80_fold_1_ep_005_val_spearman_0.39.pth
stackx_with_aux_80_fold_1_ep_006_val_spearman_0.38.pth
stackx_with_aux_80_fold_1_ep_007_val_spearman_0.38.pth
stackx_with_aux_80_fold_1_ep_008_val_spearman_0.38.pth
stackx_with_aux_80_fold_2_ep_001_val_spearman_0.37.pth
stackx_with_aux_80_fold_2_ep_002_val_spearman_0.40.pth
stackx_wit

In [10]:
!cp uploaded/stackx-bert-base/stackx-base-cased-config.json /storage/monty/quest_challenge/submits/stackx_80_aux_ep_003_submit/

In [12]:
!ls /storage/monty/quest_challenge/submits/stackx_80_aux_ep_004_submit/

stackx-base-cased-config.json
stackx-base-cased-vocab.txt
stackx_with_aux_80_fold_0_ep_004_val_spearman_0.40.pth
stackx_with_aux_80_fold_1_ep_004_val_spearman_0.40.pth
stackx_with_aux_80_fold_2_ep_004_val_spearman_0.39.pth
stackx_with_aux_80_fold_3_ep_004_val_spearman_0.39.pth
stackx_with_aux_80_fold_4_ep_004_val_spearman_0.38.pth
